In [2]:
import requests 

from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from time import sleep
import pandas as pd
import numpy as np

# Para visualizar el progreso:
import tqdm

# FUNCIONES SCRAPING:
### nota--> esto se podría haber hecho en una única función donde, a medida que se va pasando por cada página, se extrae la información de cada producto.

- URLS

In [5]:
def iniciar_scraper(user_agent, elementos_busqueda):
    
    # Options:
    opts = Options()
    opts.add_argument("--disable-extensions")
    opts.add_argument(f"User-Agent={user_agent}")
    
    # Initialize:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    sleep(5)
    
    urls_paginas_perfumes = []
    
    inicio = 1
    for elemento in elementos_busqueda:
    
        scrape_url = f"https://www.druni.es/perfumes/{elemento}"   

        browser.get(scrape_url)
        sleep(5)###########################################################################
        
        #Acepto cookies:
        if inicio == 1:            
            browser.find_element(By.CLASS_NAME, "cn-buttons").click()
            sleep(6)########################################################################

        urls_paginas_perfumes.append(scrape_url)

        while True:

            if not browser.current_url == scrape_url:
                browser.get(scrape_url)
                sleep(5)

            soup = BeautifulSoup(browser.page_source, "html.parser")

            try:
                scrape_url = soup.find("ul", class_= "items pages-items").find_all("li")[-1].find("a")["href"]
                urls_paginas_perfumes.append(scrape_url)
                sleep(1)
            except:
                break
                
        inicio += 1
    ############################################################################################################    
    browser.close()    
    return urls_paginas_perfumes



- FUNCIÓN INFO:

In [1]:
def scrape_info(url):
    
    response = requests.get(url)
    sleep(10)
    soup = BeautifulSoup(response.text, "html.parser")
    
    productos = soup.find_all("div", class_= "product-item-info")
    
    cat = soup.find("h1", class_= "page-title").find("span", class_= "base").text
    
    info_productos = []
    
    for producto in productos:
        
        categoria = cat
        
        try:
            marca = producto.find("p", class_= "product-brand").text
        except:
            marca = "Sin marca"
            
        try:
            nombre = producto.find("strong", class_= "product name product-item-name").text.strip()
        except:
            nombre = "Sin nombre"
        
        try:
            tipo = producto.find("div", class_= "product description product-item-description").text.strip()
        except:
            tipo = "Sin categoria"           
            
        try:
            precio_sin_desc = producto.find("span", class_= "price-container tax weee").find("span", class_= "price-wrapper").text.replace("\xa0", "")
        except:
            try:
                precio_sin_desc = producto.find("div", class_= "special-old-box").find("span", class_= "price-wrapper").text.replace("\xa0", "")
            except:
                precio_sin_desc = "Sin precio"
            
        try:
            descuento = producto.find("div", class_= "discount-box old-price sly-regular-price no-display").text.strip().replace("-", "")
        except:
            descuento = "No aparece"
            
        try:
            precio_con_desc = producto.find("div", class_= "price-box").find("p", class_="minimal-price").find("span", class_="price-wrapper").text.replace("\xa0", "")
        except:
            try:
                precio_con_desc = producto.find("span", class_= "special-price").find("span", class_= "price-wrapper").text.replace("\xa0", "")
            except:
                precio_con_desc = "Sin precio"
            
            
        try:
            valoracion = producto.find("div", class_= "rating-result").text.replace("\n", "")
        except:
            valoracion = "Sin rating"
            
        try:
            num_valoraciones = producto.find("a", class_= "action view").text.replace("(", "").replace(")", "")
        except:
            num_valoraciones = "Sin opiniones"
            
        info_productos.append([categoria, marca, nombre, tipo, precio_sin_desc, descuento, precio_con_desc, valoracion, num_valoraciones])

    return info_productos   

In [13]:
######################################################################################################################################################

In [14]:
######################################################################################################################################################

# Scrapeo URLS

In [4]:
# PONER EL USER-AGENT Y LOS ELEMENTOS DE BUSQUEDA:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

busqueda = ["hombre", "mujer"]

In [7]:
# Inicio el scrapeo de las url's, guardándolas en la siguiente variable

urls_totales = iniciar_scraper(user_agent, busqueda)

In [8]:
df_urls_perfumes = pd.DataFrame(urls_totales, columns= ["url"])

## CSV:

In [9]:
df_urls_perfumes.to_csv("urls_perfumes_druni.csv", index="False", sep= ",")

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################

# Scrapeo la info de todos los productos de cada página:

In [183]:
info_productos = []

for url in tqdm.tqdm(df_urls_perfumes["url"].values):

    info = scrape_info(url)
    info_productos.extend(info)

100%|██████████| 59/59 [10:25<00:00, 10.60s/it]


In [184]:
df_info_productos = pd.DataFrame(info_productos, columns= ["categoria", "marca", "nombre", "tipo", "precio_sin_desc", "descuento", "precio_con_desc", "valoracion", "num_valoraciones"] )

## CSV:

In [185]:
# df_info_productos.to_csv("datos_perfumes_druni.csv", index="False", sep= ",")

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################